In [9]:
# Load libraries + Set simulation seed
if (!require(pacman)) install.packages("pacman")
library(pacman) ; p_load('tidyverse','stargazer')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


stargazer installed



In [10]:
N <- 5000

data <- tibble(
  # ~30% of firms are in the tech sector
  tech = rbinom(N, 1, .3)) %>%
  # ~50% of tech firms are engaged in IP litigation.
  # ~10% of non-tech firms are engaged in IP litigation.
  mutate(IP = as.numeric(
    runif(N) > .9 - .4*tech)) %>%

  # Tech firms are very profitable...
  # ...but IP litigation negatively affects them.
  mutate(profit = 2 * tech
         - .3 * IP
         + rnorm(N, mean = 5))

In [11]:
summary(data)

      tech              IP             profit      
 Min.   :0.0000   Min.   :0.0000   Min.   : 1.273  
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.: 4.577  
 Median :0.0000   Median :0.0000   Median : 5.420  
 Mean   :0.2982   Mean   :0.2244   Mean   : 5.521  
 3rd Qu.:1.0000   3rd Qu.:0.0000   3rd Qu.: 6.400  
 Max.   :1.0000   Max.   :1.0000   Max.   :10.281  

In [12]:
head(data)

tech,IP,profit
<int>,<dbl>,<dbl>
0,0,3.676511
0,0,5.156389
0,0,5.140488
0,0,6.244123
0,0,5.510359
1,0,5.558361


In [13]:
# Correlation between profits and IT litigation
m1 <- lm(profit ~ IP, data)

In [14]:
# Controlling for tech sector
m2 <- lm(profit ~ IP + tech, data)

In [15]:
stargazer(m1, m2, type = "text" )


                                     Dependent variable:                 
                    -----------------------------------------------------
                                           profit                        
                               (1)                        (2)            
-------------------------------------------------------------------------
IP                          0.656***                   -0.324***         
                             (0.044)                    (0.038)          
                                                                         
tech                                                   2.028***          
                                                        (0.035)          
                                                                         
Constant                    5.373***                   4.988***          
                             (0.021)                    (0.017)          
                                     

In [16]:
# Residualize outcome
data$R_profit <- lm(profit ~ tech, data)$residuals

In [17]:
# Residualize treatment
data$R_IP <- lm(IP ~ tech, data)$residuals

In [19]:
# Correlation of residuals = causal effect
corr_resids <- lm(R_profit ~ R_IP, data)

In [20]:
stargazer(m1, m2, corr_resids,  type = "text" )


                                                 Dependent variable:                              
                    ------------------------------------------------------------------------------
                                           profit                                 R_profit        
                               (1)                        (2)                       (3)           
--------------------------------------------------------------------------------------------------
IP                          0.656***                   -0.324***                                  
                             (0.044)                    (0.038)                                   
                                                                                                  
tech                                                   2.028***                                   
                                                        (0.035)                                   
         